# Análise Exploratória Inicial e Engenharia de Atributos

### Principais problemas de negócio
- Previsão de vendas de empresa varejista
- Previsão de custos e margem líquida
- Verficiar projeção de crescimento
- Categorias e produtos em alta, bem como em baixa
- Projeção de lucro e custos por estado 
- Horas com maiores volumes de vendas
- Dia da semana com maiores volumes de vendas

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re

In [22]:
# Importação do dataset
data = pd.read_csv('../data/dados_vendas.csv', sep=',')

In [32]:
data.sample(10)

,Order Date,Order ID,Produto,Product_ean,Categoria,Endereco_Entrega,Quantidade_Comprada,Preco_Unitario,Custo_Produto,Valor_Total,Margem
54549,2019-04-11 19:46:00,193561,iPhone,7.701873e+12,Alimentation,"836 Ridge St, New York City, NY 10001",1,700.00,231.0000,700.00,469.0000
161898,2019-12-16 15:45:00,296562,Apple Airpods Headphones,2.325549e+12,Électronique,"748 Spruce St, San Francisco, CA 94016",1,150.00,97.5000,150.00,52.5000
8541,2019-01-14 01:20:00,149411,27in FHD Monitor,2.360165e+12,Sports,"648 12th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
34967,2019-03-21 20:27:00,174767,AAA Batteries (4-pack),5.714146e+12,Vêtements,"815 Highland St, New York City, NY 10001",1,2.99,1.4950,2.99,1.4950
22264,2019-03-10 14:41:00,162572,Lightning Charging Cable,4.959286e+12,Électronique,"732 Highland St, Boston, MA 02215",1,14.95,7.4750,14.95,7.4750
139599,2019-10-30 21:58:00,275170,Lightning Charging Cable,3.776762e+12,Alimentation,"96 Pine St, San Francisco, CA 94016",1,14.95,7.4750,14.95,7.4750
37607,2019-04-05 19:30:00,177308,AA Batteries (4-pack),1.845426e+12,Électronique,"944 Washington St, Boston, MA 02215",2,3.84,1.9200,7.68,3.8400
143152,2019-10-12 19:44:00,278574,34in Ultrawide Monitor,8.925686e+12,Électronique,"13 1st St, Austin, TX 73301",1,379.99,125.3967,379.99,254.5933
112030,2019-09-24 23:02:00,248697,Bose SoundSport Headphones,8.512314e+12,Vêtements,"287 Wilson St, Austin, TX 73301",1,99.99,49.9950,99.99,49.9950
106000,2019-08-29 11:38:00,242917,Lightning Charging Cable,2.118703e+12,Vêtements,"872 Highland St, Seattle, WA 98101",1,14.95,7.4750,14.95,7.4750


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185950 entries, 0 to 185949
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order Date        185950 non-null  object 
 1   Order ID          185950 non-null  int64  
 2   Product           185950 non-null  object 
 3   Product_ean       185950 non-null  float64
 4   catégorie         185950 non-null  object 
 5   Purchase Address  185950 non-null  object 
 6   Quantity Ordered  185950 non-null  int64  
 7   Price Each        185950 non-null  float64
 8   Cost price        185950 non-null  float64
 9   turnover          185950 non-null  float64
 10  margin            185950 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 15.6+ MB


In [25]:
data.nunique()

Order Date          142395
Order ID            178437
Product                 19
Product_ean         185950
catégorie                4
Purchase Address    140787
Quantity Ordered         9
Price Each              17
Cost price              17
turnover                53
margin                  54
dtype: int64

In [15]:
data.isna().sum()

Order Date          0
Order ID            0
Product             0
Product_ean         0
catégorie           0
Purchase Address    0
Quantity Ordered    0
Price Each          0
Cost price          0
turnover            0
margin              0
dtype: int64

In [26]:
data.columns

Index(['Order Date', 'Order ID', 'Product', 'Product_ean', 'catégorie',
       'Purchase Address', 'Quantity Ordered', 'Price Each', 'Cost price',
       'turnover', 'margin'],
      dtype='object')

In [28]:
# Renomeando as colunas do DF

nomes_colunas = {
 'Product': 'Produto',
 'catégorie': 'Categoria',
 'Purchase Address': 'Endereco_Entrega',
 'Quantity Ordered': 'Quantidade_Comprada',
 'Price Each': 'Preco_Unitario',
 'Cost price': 'Custo_Produto',
 'turnover': 'Valor_Total',
 'margin': 'Margem'
}

data.rename(columns = nomes_colunas, inplace= True)
data.columns

Index(['Order Date', 'Order ID', 'Produto', 'Product_ean', 'Categoria',
       'Endereco_Entrega', 'Quantidade_Comprada', 'Preco_Unitario',
       'Custo_Produto', 'Valor_Total', 'Margem'],
      dtype='object')

In [36]:
data['Produto'].value_counts()

Produto
USB-C Charging Cable          21903
Lightning Charging Cable      21658
AAA Batteries (4-pack)        20641
AA Batteries (4-pack)         20577
Wired Headphones              18882
Apple Airpods Headphones      15549
Bose SoundSport Headphones    13325
27in FHD Monitor               7507
iPhone                         6842
27in 4K Gaming Monitor         6230
34in Ultrawide Monitor         6181
Google Phone                   5525
Flatscreen TV                  4800
Macbook Pro Laptop             4724
ThinkPad Laptop                4128
20in Monitor                   4101
Vareebadd Phone                2065
LG Washing Machine              666
LG Dryer                        646
Name: count, dtype: int64

#### Traduzino os nomes dos 19 Produtos, para melhor visualização

In [37]:
muda_nome_produtos = {
 'USB-C Charging Cable': 'Cabo de Carregamento USB-C',
 'Lightning Charging Cable': 'Cabo de Carregamento Turbo',
 'AAA Batteries (4-pack)': 'Pilhas AAA (4 Unidades)',
 'AA Batteries (4-pack)': 'Pilhas AA (4 Unidades)',
 'Wired Headphones': 'Fone de Ouvido Sem Fio',
 'Apple Airpods Headphones': 'Apple Airpods',
 'Bose SoundSport Headphones': 'Fone de Ouvido Esportivo Bose',
 '27in FHD Monitor': 'Monitor 27" Full HD',
 'iPhone': 'iPhone',
 '27in 4K Gaming Monitor': 'Monitor gamer 27" 4K',
 '34in Ultrawide Monitor': 'Monitor Ultrawide 34"',
 'Google Phone': 'Google Phone',
 'Flatscreen TV': 'TV Tela Plana',
 'Macbook Pro Laptop': 'Notebook Macbook Pro',
 'ThinkPad Laptop': 'Notebook ThinkPad',
 '20in Monitor': 'Monitor 20"',
 'Vareebadd Phone': 'Smartphone Vareebadd',
 'LG Washing Machine': 'Máquina de Lavar Roupas LG',
 'LG Dryer': 'Secador LG'
}

In [38]:
# Mapeando a coluna 
data['Produto'] = data['Produto'].map(muda_nome_produtos)

#### Mudando nome de Categoria

In [42]:
muda_nome_cat = {
 'Sports': 'Esportes',
 'Vêtements': 'Roupas',
 'Alimentation': 'Alimentação',
 'Électronique': 'Eletrônica'
 }

In [43]:
# Mapeando a coluna Categoria
data['Categoria'] = data['Categoria'].map(muda_nome_cat)

In [45]:
data.sample(5)

,Order Date,Order ID,Produto,Product_ean,Categoria,Endereco_Entrega,Quantidade_Comprada,Preco_Unitario,Custo_Produto,Valor_Total,Margem
11996,2019-02-23 15:15:00,152726,Pilhas AAA (4 Unidades),7.579094e+12,Alimentação,"89 5th St, San Francisco, CA 94016",2,2.99,1.495,5.98,2.990
162416,2019-12-17 08:43:00,297056,iPhone,5.643222e+12,Eletrônica,"45 Willow St, New York City, NY 10001",1,700.00,231.000,700.00,469.000
167564,2019-12-31 11:34:00,301994,Cabo de Carregamento USB-C,5.793837e+12,Roupas,"104 West St, Atlanta, GA 30301",1,11.95,5.975,11.95,5.975
182768,2019-12-08 18:45:00,316605,Fone de Ouvido Esportivo Bose,7.086864e+12,Roupas,"926 Sunset St, Seattle, WA 98101",1,99.99,49.995,99.99,49.995
119110,2019-09-30 21:05:00,255500,Cabo de Carregamento Turbo,5.144911e+12,Esportes,"33 Jackson St, Los Angeles, CA 90001",1,14.95,7.475,14.95,7.475


## Primeiras Decisões 

- Vou excluir a coluna "Product_ean", pois não representa informação que será válida para o problema,
- Vou excluir a coluna "Order ID", pois também não representa informação útil,
- Na coluna "Endereço_Entrega", irei deixar apenas o nome do estado
- A coluna "Order Date" será excluída e dará 'vida' a 3 novas colunas, são elas:
  - "Horario": Representando o horário da venda
  - "Data": Representando o dia da venda
  - "Dia_Semana": Representando o dia da semana em que a compra foi realizada
- A coluna "Custo_Produto" não está calculando o custo pela quantidade comprada, apenas pela quantidade unitária, farei essa correção também

In [46]:
data = data.drop('Product_ean', axis= 1)
data = data.drop('Order ID', axis= 1)

In [95]:
data.sample(5)

,Order Date,Produto,Categoria,Endereco_Entrega,Quantidade_Comprada,Preco_Unitario,Custo_Produto,Valor_Total,Margem,Estado,Cidade
74125,2019-06-16 08:12:00,Pilhas AA (4 Unidades),Alimentação,"686 Washington St, Dallas, TX 75001",1,3.84,1.920,3.84,1.920,TX,Dallas
180060,2019-12-14 17:07:00,Cabo de Carregamento Turbo,Eletrônica,"166 Dogwood St, Dallas, TX 75001",1,14.95,7.475,14.95,7.475,TX,Dallas
153938,2019-11-14 21:46:00,Pilhas AA (4 Unidades),Alimentação,"196 North St, Boston, MA 02215",1,3.84,1.920,3.84,1.920,MA,Boston
46607,2019-04-02 17:19:00,Fone de Ouvido Sem Fio,Esportes,"617 10th St, San Francisco, CA 94016",1,11.99,5.995,11.99,5.995,CA,San Francisco
139938,2019-10-10 21:34:00,TV Tela Plana,Roupas,"881 West St, Boston, MA 02215",1,300.00,99.000,300.00,201.000,MA,Boston


In [64]:
frase = '941 Jefferson St, Boston, MA 02215'

# Usando regex para encontrar o estado (TX)
padrao = r',\s*([A-Z]{2})\s+\d{5}$'
correspondencia = re.search(padrao, frase)

correspondencia[1]

'MA'

#### Utilizando REGEX para criar novas colunas, com informações de Estado e Cidade

In [70]:
# Funçao para nome de estado 
def nome_estado(str):
 padrao_re = r',\s*([A-Z]{2})\s+\d{5}$'
 estado = re.search(padrao_re, str)

 return estado[1]

In [71]:
# Aplicando a função e nova coluna
data['Estado'] = data['Endereco_Entrega'].map(nome_estado)

In [87]:
frase = '55 Willow St, Atlanta, GA 30301'

# Usando regex para encontrar o nome da cidade (assume que a cidade segue o estado)
padrao = r',\s*([A-Z][a-z\s]+),\s*[A-Z]{2}'
correspondencia = re.search(padrao, frase)

correspondencia[1]

'Atlanta'

In [92]:
# Função para nome de cidade
def nome_cidade(x):
 padrao = r',\s*([^,]+),\s*[A-Z]{2}'
 cidade = re.search(padrao, x)

 return cidade[1]

In [93]:
data['Cidade'] = data['Endereco_Entrega'].map(nome_cidade)

In [96]:
# Como já crei as colunas cidade e estado, excluirei a coluna Endereço_Entrega
data = data.drop('Endereco_Entrega', axis= 1)

In [98]:
data.sample(5)

,Order Date,Produto,Categoria,Quantidade_Comprada,Preco_Unitario,Custo_Produto,Valor_Total,Margem,Estado,Cidade
18516,2019-02-10 15:26:00,Pilhas AAA (4 Unidades),Alimentação,1,2.99,1.495,2.99,1.495,NY,New York City
159102,2019-11-01 20:28:00,Apple Airpods,Eletrônica,1,150.00,97.500,150.00,52.500,CA,San Francisco
168035,2019-12-02 12:53:00,Apple Airpods,Alimentação,1,150.00,97.500,150.00,52.500,CA,San Francisco
48253,2019-04-09 22:21:00,Apple Airpods,Roupas,1,150.00,97.500,150.00,52.500,CA,Los Angeles
170739,2019-12-12 15:58:00,Fone de Ouvido Esportivo Bose,Alimentação,1,99.99,49.995,99.99,49.995,MA,Boston
